In [1]:
import numpy as np
from utils.dataloader import *
from utils.coordinate_calibration import PointCloudRegistration
from utils.meshgrid import Meshgrid
from utils.icp import IterativeClosestPoint, Matching

In [2]:
RIGID_BODY_DATA_A = f'./pa345_data/Problem3-BodyA.txt'
RIGID_BODY_DATA_B = f'./pa345_data/Problem3-BodyB.txt'

SURFACE_DATA = f'./pa345_data/Problem3MeshFile.sur'
# SURFACE_DATA = f'./pa345_data/test.sur'

SAMPLE_DATA = f'./pa345_data/PA3-B-Debug-SampleReadingsTest.txt'

rigidbody_dl_A = RigidBodyDataloader.read_file(RIGID_BODY_DATA_A)
rigidbody_dl_B = RigidBodyDataloader.read_file(RIGID_BODY_DATA_B)

# print("rigid body")
# print(rigidbody_dl_A.markers)
# print(rigidbody_dl_A.tip)

surface_dl = Surfaceloader.read_file(SURFACE_DATA)
# print("surface")
# print(surface_dl.N_vertices)
# print(surface_dl.N_triangles)
# print(surface_dl.vertices.shape)
# print(surface_dl.triangles.shape)

sample_dl = SampleReadingsDataloader.read_file(SAMPLE_DATA, delimiter=',', N_A=rigidbody_dl_A.N_markers, N_B=rigidbody_dl_B.N_markers)
# print("sample A", sample_dl.body_A)
# print("sample B", sample_dl.body_B)
# print("dummy", sample_dl.dummy)

### Get F<sub>A, k</sub> and F<sub>B, k</sub>

In [3]:
# TODO get F_A
rigidbody_dl_A_markers = rigidbody_dl_A.markers.reshape(1, -1, 3) # markers of body A in body A coordinates

sample_dl_A = sample_dl.body_A # samples of body A markers in tracker coordinates
num_samples = sample_dl.N_samps

# perform registration for each frame
reg_F_A = PointCloudRegistration()
F_A = []
for i in range(num_samples):
    sample_dl_A_i = sample_dl_A[i].reshape(1, -1, 3)
    F_A_i, err = reg_F_A.register(rigidbody_dl_A_markers, sample_dl_A_i)
    F_A.append(F_A_i)

F_A = np.array(F_A)

# TODO get F_B
ridigbody_dl_B_markers = rigidbody_dl_B.markers.reshape(1, -1, 3) # markers of body B in body B coordinates
sample_dl_B = sample_dl.body_B # samples of body B markers in tracker coordinates

# perform registration for each frame
reg_F_B = PointCloudRegistration()
F_B = []
for i in range(num_samples):
    sample_dl_B_i = sample_dl_B[i].reshape(1, -1, 3)

    F_B_i, err = reg_F_B.register(ridigbody_dl_B_markers, sample_dl_B_i)
    F_B.append(F_B_i)

F_B = np.array(F_B)

Performing 3D point cloud registration...


100ep [00:00<00:00, 349.67it/s, err=0.00281]


Done.
Performing 3D point cloud registration...


100ep [00:00<00:00, 979.95it/s, err=0.00266]


Done.
Performing 3D point cloud registration...


...

Done.
Performing 3D point cloud registration...


100ep [00:00<00:00, 1000.43it/s, err=0.00226]


Done.
Performing 3D point cloud registration...


100ep [00:00<00:00, 654.20it/s, err=0.00226]


Done.


### Get d<sub>k

In [4]:
A_tip = rigidbody_dl_A.tip
A_tip = np.append(A_tip, 1) # add 1 for homogenous coordinates
d_k = []

for i in range(num_samples):
    F_A_i = F_A[i] # get F_A for frame i
    F_B_i_inv = np.linalg.inv(F_B[i]) # get F_B inverse for frame i
    # print(A_tip.shape)
    # print(F_A_i.shape)
    # print(F_B_i_inv.shape)

    d_k.append(F_B_i_inv @ F_A_i @ A_tip) # d_k = F_B^-1 * F_A * A_tip

d_k = np.array(d_k)[:,:3]
print(d_k.shape)
print(d_k)

(15, 3)
[[-37.64833678  -9.48581429 -12.76682898]
 [-18.60510135  -9.65676445  -4.29505254]
 [ -2.14008856  -7.91886808  52.44877509]
 [ 17.87241499 -12.57782081   1.66919045]
 [-13.95441346  21.50282627  25.2969365 ]
 [ 28.96426532  19.0550331  -13.73958009]
 [ 24.2614743   22.12267255  -1.99781554]
 [-27.75457871 -23.80052244 -48.36744324]
 [-28.46403537   7.90253067 -35.89238329]
 [-12.65584902   9.56599089  12.90058673]
 [-27.11807733 -28.65576806 -14.35566804]
 [ 37.25503378   5.25779861   4.71164992]
 [ 20.17629056 -14.87243941  -4.87098927]
 [ -4.2582231   12.83878122  54.08074924]
 [ 32.08328176  15.8541175   -2.09464545]]


In [5]:
icp = IterativeClosestPoint(Matching.SIMPLE_LINEAR)

# print(surface_dl.triangles)
# print(surface_dl.vertices)
print(surface_dl.triangles.shape)


mesh = Meshgrid(surface_dl.vertices, surface_dl.triangles)


# print(surface_dl.vertices[:20])
# print(mesh.trangles[-2:])
pt, dist = icp.match(d_k, mesh)

print(np.around(pt, 2))
print(dist)

(3135, 3)
(array([-20.110569, -16.44264 , -48.20013 ]), array([-23.977119, -12.596711, -48.630154]), array([-20.396912, -12.622053, -49.394562]))
[-37.64833678  -9.48581429 -12.76682898]
[38.50659626] [[-23.977119 -12.596711 -48.630154]]
(array([-20.145435, -12.847064, -46.920708]), array([-20.110569, -16.44264 , -48.20013 ]), array([-20.396912, -12.622053, -49.394562]))
[-37.64833678  -9.48581429 -12.76682898]
[38.52449899] [[-20.145435 -12.847064 -46.920708]]
(array([-20.052992,  -9.161464, -48.999603]), array([-16.576967,  -5.792709, -47.751953]), array([-15.800093, -10.141426, -48.594055]))
[-37.64833678  -9.48581429 -12.76682898]
[40.28045772] [[-20.052992  -9.161464 -48.999603]]
(array([-12.928831,  -9.491612, -47.82164 ]), array([-15.800093, -10.141426, -48.594055]), array([-16.576967,  -5.792709, -47.751953]))
[-37.64833678  -9.48581429 -12.76682898]
[41.00732317] [[-16.576967  -5.792709 -47.751953]]
(array([-20.032549, -25.838306, -43.115608]), array([-23.855099, -23.710838, -